In [1]:
using JuMP,HiGHS,LaTeXStrings, GLPK, StatsPlots, Plots, DelimitedFiles

# Input Data

In [2]:
#Input data

asset_quantity = readdlm("data/sample_asset_quantity.csv", ',', Float64)
asset_tiers = readdlm("data/sample_asset_tiers.csv", ',', Float64)
asset_value = readdlm("data/sample_asset_value.csv", ',', Float64)

account_exposure = readdlm("data/sample_account_exposure.csv", ',', Float64)
account_durations = readdlm("data/sample_account_duration.csv", ',', Float64)


haircuts = readdlm("data/sample_haircuts.csv", ',', Float64)/100


single_limits = readdlm("data/sample_single_limits.csv", ',', Float64)

10×5 Matrix{Float64}:
  20997.7            1.53498e5   27959.6          4957.23        12027.8
 225485.0        47932.1          8730.8          5159.9        225485.0
 634872.0            7.89539e5  143814.0        634872.0        634872.0
  58894.0        33420.1          6087.46        35976.9         58894.0
   6623.8        48421.5          8819.95         1563.78         3794.22
      1.6351e5       1.1953e6        2.17723e5       3.86022e5   93661.2
 558000.0       186000.0             1.22826e5   92915.7        930000.0
      3.576e6        1.192e6    783709.0             5.9286e5        5.96e6
 259800.0        86600.0        433000.0             5.9286e6   433000.0
      6.24e7         2.08e6          1.49949e6       1.13434e6       1.04e7

In [3]:
# Auxillary variables

tier_set = [0.2, 0.5, 0.8]

n_assets = length(asset_tiers)

n_accounts = length(account_durations)



cost_factor_matrix = Array{Float64}(undef, length(asset_tiers), length(account_durations))

for i in 1:n_assets
    for j in 1:n_accounts
        cost_factor_matrix[i,j] = abs(asset_tiers[i] - account_durations[j])
    end
end

cost_factor_matrix = round.(cost_factor_matrix, digits = 2);

print(cost_factor_matrix)




[

0.2

 0.2 0.8 0.8 0.8; 0.2 0.2 0.8 0.8 0.8; 0.2 0.2 0.8

 0.8 0.8; 0.2 0.2 0.8 0.8 0.8; 0.5 0.5 0.5 0.5 0.5; 0.5 

0.5 0.5 0.5 0.5; 0.8 0.8 0.2 0.2 0.2; 0.8 0.8 0.2 0.2 0.2; 0.8 0.8 

0.2 0.2 0.2; 0.8 0.8 0.2 0.2 0.2]

# Constructing JuMP model

In [4]:

model = Model()

#Q 
@variable(model, Q[i = 1:n_assets, j = 1:n_accounts], lower_bound = 0, upper_bound = 1)


#Consistency constraint
@constraint(model,[i = 1:n_assets],sum(Q[i,j] for j in 1:n_accounts) <= 1)

#Requirement constraint
@constraint(model, [j = 1:n_accounts],
    sum(Q[i,j]*asset_quantity[i]*asset_value[i]*haircuts[i,j] for i in 1:n_assets )
    >= account_exposure[j])


#Single Limits
@constraint(model, [i = 1:n_assets, j =1:n_accounts], 
    Q[i,j]*asset_quantity[i] <= single_limits[i,j])

#Objective_function
@objective(model, Min, sum(Q[i,j]*cost_factor_matrix[i,j] for i in 1:n_assets,j in 1:n_accounts));
print(model)

Min 0.2 Q[1,1] + 0.2 Q[1,2] + 0.8 Q[1,3] + 0.8 Q[1,4] + 0.8 Q[1,5] + 0.2 Q[2,1] + 0.2 Q[2,2] + 0.8 Q[2,3] + 0.8 Q[2,4] + 0.8 Q[2,5] + 0.2 Q[3,1] + 0.2 Q[3,2] + 0.8 Q[3,3] + 0.8 Q[3,4] + 0.8 Q[3,5] + 0.2 Q[4,1] + 0.2 Q[4,2] + 0.8 Q[4,3] + 0.8 Q[4,4] + 0.8 Q[4,5] + 0.5 Q[5,1] + 0.5 Q[5,2] + 0.5 Q[5,3] + 0.5 Q[5,4] + 0.5 Q[5,5] + 0.5 Q[6,1] + 0.5 Q[6,2] + 0.5 Q[6,3] + 0.5 Q[6,4] + 0.5 Q[6,5] + 0.8 Q[7,1] + 0.8 Q[7,2] + 0.2 Q[7,3] + 0.2 Q[7,4] + 0.2 Q[7,5] + 0.8 Q[8,1] + 0.8 Q[8,2] + 0.2 Q[8,3] + 0.2 Q[8,4] + 0.2 Q[8,5] + 0.8 Q[9,1] + 0.8 Q[9,2] + 0.2 Q[9,3] + 0.2 Q[9,4] + 0.2 Q[9,5] + 0.8 Q[10,1] + 0.8 Q[10,2] + 0.2 Q[10,3] + 0.2 Q[10,4] + 0.2 Q[10,5]
Subject to


 337580.7187721691 Q[1,1] + 1.416835560987015e6 Q[2,1] + 242181.403673832 Q[3,1] + 530751.1039390559 Q[4,1] + 1.9252052101651952e6 Q[5,1] + 2.48391407769804e6 Q[6,1] + 98653.12350377608 Q[7,1] + 990856.6525174977 Q[8,1] + 146168.41998874999 Q[9,1] + 90366.7778 Q[10,1] >= 824000.0
 340990.62502580293 Q[1,2] + 1.416835560987015e6 Q[2,2] + 242181.403673832 Q[3,2] + 482501.0035857849 Q[4,2] + 1.9991746731706962e6 Q[5,2] + 2.509004118911999e6 Q[6,2] + 103845.39316186956 Q[7,2] + 1.0430070026499976e6 Q[8,2] + 153861.494725 Q[9,2] + 90593.26095419283 Q[10,2] >= 669292.0
 340990.62502580293 Q[1,3] + 1.288032328182894e6 Q[2,3] + 242181.403673832 Q[3,3] + 482501.0035857849 Q[4,3] + 1.9446517274590374e6 Q[5,3] + 2.509004118911999e6 Q[6,3] + 101809.20898609381 Q[7,3] + 1.0225558849898352e6 Q[8,3] + 153861.494725 Q[9,3] + 93257.76863099477 Q[10,3] >= 548600.0
 353858.19577229326 Q[1,4] + 1.2764284333203589e6 Q[2,4] + 242181.403673832 Q[3,4] + 478154.14769236505 Q[4,4] + 2.0180348114616266e6 Q[5,4]

225485 Q[2,1] <= 225485.0
 634872 Q[3,1] <= 634872.0
 58894 Q[4,1] <= 58894.0
 34391 Q[5,1] <= 6623.804430129181
 10953220 Q[6,1] <= 163510.24427398635
 929999.959 Q[7,1] <= 557999.9754
 5.959999998e6 Q[8,1] <= 3.5759999987999997e6
 433000 Q[9,1] <= 259800.0
 1040000 Q[10,1] <= 6.24e7
 191165.7 Q[1,2] <= 153498.09678305002


 225485 Q[2,2] <= 47932.08883865839
 634872 Q[3,2] <= 789538.8947300112
 58894 Q[4,2] <= 33420.1422097028
 34391 Q[5,2] <= 48421.52164059488
 10953220 Q[6,2] <= 1.1952971913781841e6
 929999.959 Q[7,2] <= 185999.99180000002
 5.959999998e6 Q[8,2] <= 1.1919999996e6
 433000 Q[9,2] <= 86600.0
 1040000 Q[10,2] <= 2.08e6
 191165.7 Q[1,3] <= 27959.580470500918
 225485 Q[2,3] <= 8730.79942416364
 634872 Q[3,3] <= 143814.0063260494
 58894 Q[4,3] <= 6087.4575973957735
 34391 Q[5,3] <= 8819.949297011817
 10953220 Q[6,3] <= 217722.62138036144
 

929999.959 Q[7,3] <= 122826.33874574634
 5.959999998e6 Q[8,3] <= 783709.023667984
 433000 Q[9,3] <= 433000.0
 1040000 Q[10,3] <= 1.4994913318686462e6
 191165.7 Q[1,4] <= 4957.233617419813
 225485 Q[2,4] <= 5159.902459680712
 634872 Q[3,4] <= 634872.0
 58894 Q[4,4] <= 35976.87440060902
 34391 Q[5,4] <= 1563.7770103601952
 10953220 Q[6,4] <= 386022.2077073808
 929999.959 Q[7,4] <= 92915.66873438218
 5.959999998e6 Q[8,4] <= 592860.2022243565
 433000 Q[9,4] <= 5.928602022243565e6
 1040000 Q[10,4] <= 1.1343352027319933e6
 191165.7 Q[1,5] <= 12027.818066055808
 225485 Q[2,5] <= 225485.0
 634872 Q[3,5] <= 634872.0
 58894 Q[4,5] <= 58894.0
 34391 Q[5,5] <= 3794.218071627395
 10953220 Q[6,5] <= 93661.20788509898
 929999.959 Q[7,5] <= 929999.959

...


 Q[3,3] <= 1.0
 Q[4,3] <= 1.0
 Q[5,3] <= 1.0
 Q[6,3] <= 1.0
 

Q[7,3] <= 1.0
 Q[8,3] <= 1.0
 Q[9,3] <= 1.0
 Q[10,3] <= 1.0
 Q[1,4] <= 1.0
 Q[2,4] <= 1.0
 Q[3,4] <= 1.0
 Q[4,4] <= 1.0
 Q[5,4] <= 1.0
 Q[6,4] <= 1.0
 

Q[7,4] <= 1.0
 Q[8,4] <= 1.0
 Q[9,4] <= 1.0
 Q[10,4] <= 1.0
 Q[1,5] <= 1.0


 Q[2,5] <= 1.0
 Q[3,5] <= 1.0
 Q[4,5] <= 1.0
 Q[5,5] <= 1.0
 Q[6,5] <= 1.0
 Q[7,5] <= 1.0
 Q[8,5] <= 1.0
 Q[9,5] <= 1.0


 Q[10,5] <= 1.0


# Solving with HiGHs


In [5]:
set_optimizer(model, HiGHS.Optimizer)
set_optimizer_attribute(model, "solver", "simplex")
optimize!(model)

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
15 rows, 50 cols, 100 nonzeros
14 rows, 50 cols, 95 nonzeros
Presolve : Reductions: rows 14(-51); columns 50(-0); elements 95(-55)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 5(20146.4) 1s
          6     4.7458912088e-01 Pr: 0(0); Du: 0(3.59503e-08) 1s
Using EKK primal simplex solver
  Iteration        Objective     Infeasibilities num(sum)
          6     4.7458912088e-01 Pr: 0(0); Du: 1(0.000294505) 1s
          7     4.7457081130e-01 Pr: 0(0) 1s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 7
Objective value     :  4.7457081130e-01
HiGHS run time      :          1.23


In [6]:

println("Objective value is $(objective_value(model))")


Objective value is 0.4745708112951061


# Export to .csv

In [7]:
Q_values_HiGHS_simplex = zeros((n_assets, n_accounts))


for i in 1:n_assets
    for j in 1:n_accounts
        Q_values_HiGHS_simplex[i,j] = value(Q[i,j])
        if value(Q[i,j]) != 0 
            println("Q_{$(i), $(j)} = $(value(Q[i,j]))")
        end
    end
end

println(Q_values_HiGHS_simplex)
#store all Values to an array


# Uncomment line below to save to csv files

#writedlm("Q_values_HiGHS_simplex.csv", Q_values_HiGHS_simplex, ',')

Q_{2, 1} = 0.5815777234063592
Q_{2, 2} = 0.21257329240817965
Q_{5, 2} = 0.04717413067284134
Q_{5, 3} = 0.18731708267572655
Q_{6, 2} = 0.10912747040397108
Q_{6, 3} = 0.019877499162836265


Q_{6, 5} = 0.008551020420031641
Q_{8, 3} = 0.13149480267298214
Q_{8, 4} = 0.062170742703785954


Q_{8, 5} = 0.4549194869457063
[

0.0 

0.0 0.0 0.0 

0.0; 0.5815777234063592 0.21257329240817965 

0.0 0.0 0.0; 0.0 0.0 0.0

 0.0 0.0; 0.0 0.0 0.0 0.0 0.0; 0.0 0.04717413067284134 

0.18731708267572655 0.0 0.0; 0.0 0.10912747040397108 0.019877499162836265 0.0 0.008551020420031641; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.13149480267298214 0.062170742703785954 0.4549194869457063; 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0]